## 1. Install Dependencies

In [1]:
!pip install atari-py

In [2]:
!pip install tensorflow gym keras-rl2 gym[atari]

In [3]:
import gym
import random


In [4]:
env = gym.make("SpaceInvaders-v0")
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [5]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
episodes = 5

for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice(range(0,actions))
        n_state, reward, done, info = env.step(action)
        score += reward
    #end
    
    print("Episode: {}, Score: {}".format(episode, score))
#end

env.close()

Episode: 1, Score: 30.0
Episode: 2, Score: 35.0
Episode: 3, Score: 90.0
Episode: 4, Score: 120.0
Episode: 5, Score: 135.0


In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

2021-09-07 21:55:29.592237: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-09-07 21:55:29.592653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## 2. Build Model

In [8]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height,width,channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
#end

In [15]:
del model

In [16]:

model = build_model(height, width, channels, actions)

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

## 3. Build Agent with Keras-RL

In [18]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [19]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return dqn
#end

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

2021-09-07 21:55:58.913599: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1257] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-09-07 21:55:58.913636: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1263]      


Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  498/10000: episode: 1, duration: 13.900s, episode steps: 498, steps per second:  36, episode reward: 80.000, mean reward:  0.161 [ 0.000, 20.000], mean action: 2.500 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1382/10000: episode: 2, duration: 536.228s, episode steps: 884, steps per second:   2, episode reward: 160.000, mean reward:  0.181 [ 0.000, 30.000], mean action: 2.548 [0.000, 5.000],  loss: 10.972288, mean_q: 2.463008, mean_eps: 0.892810
 2183/10000: episode: 3, duration: 1015.138s, episode steps: 801, steps per second:   1, episode reward: 135.000, mean reward:  0.169 [ 0.000, 30.000], mean action: 2.479 [0.000, 5.000],  loss: 0.736484, mean_q: 2.229579, mean_eps: 0.839620
 2677/10000: episode: 4, duration: 595.324s, episode steps: 494, steps per second:   1, episode reward: 50.000, mean reward:  0.101 [ 0.000, 15.000], me

## 4. Reloading Agent From Memory